## Step 1: Training the Models

In [1]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10n.pt
!wget -P {HOME}/weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10s.pt
!wget -P {HOME}/weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10b.pt
!wget -P {HOME}/weights -q https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10x.pt
!ls -lh {HOME}/weights

total 514304
-rw-r--r--  1 richardjiang  staff    79M May 23 17:38 yolov10b.pt
-rw-r--r--  1 richardjiang  staff    11M Sep 14 23:38 yolov10n.pt
-rw-r--r--  1 richardjiang  staff    31M Sep 14 23:38 yolov10s.pt
-rw-r--r--  1 richardjiang  staff   122M May 23 17:38 yolov10x.pt


## 1.1 图片数据训练

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os

# Initialize the four YOLO models
yolov10_models = {
    "yolov10n": YOLO(f'{HOME}/weights/yolov10n.pt'),
    "yolov10s": YOLO(f'{HOME}/weights/yolov10s.pt'),
    "yolov10x": YOLO(f'{HOME}/weights/yolov10x.pt'),
    "yolov10b": YOLO(f'{HOME}/weights/yolov10b.pt')
}

# Path to the dataset configuration file (YAML)
DATASET_CONFIG = 'trafic_data/data_1.yaml'

# Directory to save the training graphs for all models
main_folder = 'training-graphs'
if not os.path.exists(main_folder):  # 自动帮我们创立好文件夹
    os.makedirs(main_folder)  # 每个模型的文件里面总共两个图（一个accuracy，一个loss）

# Number of epochs to wait for loss improvement before stopping
PATIENCE = 5

# Train each of the four models
for model_name, model in yolov10_models.items():
    print(f"Training {model_name}...")

    # Directory for saving this model's training graphs
    model_folder = os.path.join(main_folder, model_name)
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)
    
    # Train the model
    results = model.train(
        data=DATASET_CONFIG,   # path to the YAML config file
        epochs=20,             # number of training epochs
        batch=128,             # batch size
        imgsz=640,             # image size for training (640x640)
        name=model_name,       # name for this experiment run
        workers=8,             # number of data loading workers
        plots=True,            # save loss and mAP plots
        lr0=0.001,             # learning-rate (batch size 大，lr 要小)
        patience=PATIENCE,     # Early stopping: stop if no improvement in 5 epochs
        device='0',            # 0: GPU, 1: CPU
        save=True,             # Checkpoints: saves the model at each improvement
    )

    # Retrieve training metrics (loss, mAP, etc.)
    metrics = results.metrics

    # Define the range of epochs
    epochs = range(1, 21)  # Adjust according to your number of epochs

    # Loss plot
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, metrics['train/loss'], label='Training Loss')
    plt.plot(epochs, metrics['val/loss'], label='Validation Loss')
    plt.title(f'{model_name} - Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    loss_file_path = os.path.join(model_folder, f'{model_name}_loss.png')
    plt.savefig(loss_file_path)
    plt.close()

    # Accuracy (mAP) plot
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, metrics['train/map50'], label='Training mAP@0.5')
    plt.plot(epochs, metrics['val/map50'], label='Validation mAP@0.5')
    plt.title(f'{model_name} - Accuracy (mAP) Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('mAP@0.5')
    plt.legend()
    plt.grid(True)
    accuracy_file_path = os.path.join(model_folder, f'{model_name}_accuracy.png')
    plt.savefig(accuracy_file_path)
    plt.close()

    print(f"Training completed for {model_name} and plots saved in {model_folder}.")

print("All models have been trained and plots saved.")

## 1.2 视频类型训练数据

In [ ]:
video_dataset_config = r'C:\Users\ASUS\Desktop\video_dataset_name\name_of_the_file.yaml'

video_training_graph_folder = "training-graphs-video"
if not os.path.exists(video_training_graph_folder):
    os.makedirs(video_training_graph_folder)

for model_name, model in yolov10_models.items():
    print(f"Training {model_name}...")
    
    model_folder = os.path.join(video_training_graph_folder, model_name)
    if not os.path.exists(model_folder):
        os.makedirs(model_folder)
    
    video_train_results = model.train(
        data=DATASET_CONFIG,   # path to the YAML config file
        epochs=15,             # number of training epochs
        batch=128,             # batch size
        imgsz=640,             # image size for training (640x640)
        name=model_name,       # name for this experiment run
        workers=8,             # number of data loading workers
        plots=True,            # save loss and mAP plots
        lr0=0.001,             # learning-rate (batch size 大，lr 要小)
        patience=PATIENCE,     # Early stopping: stop if no improvement in 5 epochs
        device='0',            # 0: GPU, 1: CPU
        save=True,             # Checkpoints: saves the model at each improvement
    )
    epochs = range(1, 16)
    metrics = video_train_results.metrics

    # Loss vs Epochs Plot
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, metrics=['train/loss'], label='Training Loss')
    plt.plot(epochs, metrics=['val'/'loss'], label='Validation Loss')
    plt.title(f'{model_name} - Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylavel('Loss')
    plt.legend()
    plt.grid(True)
    video_loss_file_path = os.path.join(model_name, f"{model_name}_loss.png")
    plt.savefig(video_loss_file_path)
    plt.close()

    # Accuracy vs Epochs
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, metrics=['train'/'map50'], label='Train Accuracy')
    plt.plot(epochs, metrics=['val'/'map50'], label='Validation Accuracy')
    plt.title(f'{model_name} - Accuracy (mAP) Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (mAP)')
    plt.legend()
    plt.grid()
    video_accuracy_file_path = os.path.join(model_name, f"{model_name}_accuracy.png")
    plt.savefig(video_accuracy_file_path)
    plt.close()

    print(f"Successfully complete training and validation for {model_name}")

print(f"Successfully complete training and validation (video_dataset) for all models")